pip install --upgrade huggingface-hub>=0.15.1 transformers>=4.30 accelerate>=0.18 sentence-transformers>=2.2.2


In [2]:
import os
import json
from glob import glob
from sentence_transformers import SentenceTransformer
import faiss
import pickle

# 1. 加载问答数据
data_dir = r"C:\Users\15278\CHATBOT\data_crawl-20250703T175549Z-1-001\data_crawl"
json_files = glob(os.path.join(data_dir, "quora_all_scraped_*_extracted.json"))

all_texts = []

for file_path in json_files:
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)["data"]
        for item in data:
            q = item.get("question_text", "")
            for a in item.get("answer_texts", []):
                text = f"Q: {q}\nA: {a}"
                all_texts.append(text)

print(f"✅ 加载问答对数量：{len(all_texts)}")

# 2. 直接用整条文本生成向量（不做切分）
texts = all_texts

# 3. 加载中文 Embedding 模型
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 4. 嵌入 & 建立 FAISS 索引
embeddings = model.encode(texts, show_progress_bar=True, convert_to_numpy=True)
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# 5. 保存向量库和原文
with open("qa_chunks.pkl", "wb") as f:
    pickle.dump(texts, f)
faiss.write_index(index, "qa_index.faiss")

print("✅ 向量库构建完成并保存！")


C:\Users\15278\.conda\envs\chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 加载问答对数量：10088


Batches: 100%|██████████| 316/316 [00:11<00:00, 28.48it/s]

✅ 向量库构建完成并保存！


In [5]:
import faiss
import pickle
import numpy as np

# 1. 加载索引
index = faiss.read_index("qa_index.faiss")

# 2. 加载文本块（qa_chunks.pkl）
with open("qa_chunks.pkl", "rb") as f:
    qa_chunks = pickle.load(f)

# 3. 检查索引和文本块数量是否一致
assert index.ntotal == len(qa_chunks), f"索引数量 ({index.ntotal}) 和文本块数量 ({len(qa_chunks)}) 不一致"

# 4. 获取前3条向量 + 对应文本
print("索引中的前三条向量及对应文本：\n")
for i in range(10):
    vector = index.reconstruct(i)  # 获取第i条向量
    text = qa_chunks[i]            # 获取第i条对应的文本
    print(f"=== 第 {i} 条 ===")
    print(f"[向量长度]: {len(vector)}")
    print(f"[文本内容]: {text}\n")


索引中的前三条向量及对应文本：

=== 第 0 条 ===
[向量长度]: 384
[文本内容]: Q: As a Java Developer, how can I switch my career to Machine Learning?
A: What skill do you need to learn? Machine learning. If you want to skip the rant and go to my actual advice, scroll to the bottom! Rant Yes, it’s an obvious advantage that you know some programming. Coding proficiency is a basic requirement to do machine learning, although it’s not as important as in a typical software developer job. Mathematical optimization and calculus is somewhat important. Linear algebra and statistics is really important. Probability and combinatorics is really important. But these are just prerequisites. Most importantly, machine learning is an entire branch of computer science Continue ReadingWhat skill do you need to learn? Machine learning. If you want to skip the rant and go to my actual advice, scroll to the bottom! Rant Yes, it’s an obvious advantage that you know some programming. Coding proficiency is a basic requirement to do mach

In [9]:
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

# 1. Load vector index and texts
index = faiss.read_index("qa_index.faiss")
with open("qa_chunks.pkl", "rb") as f:
    texts = pickle.load(f)  # Make sure texts is a list of strings

# 2. Load English embedding model
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 3. Load local Qwen2 model (4bit)
model_name = "unsloth/qwen2-1.5b-bnb-4bit"
max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True
)
model.eval()

# 4. Main QA function
def answer_question(query, top_k=3):
    # 4.1 Encode query
    query_vec = embedder.encode([query], convert_to_numpy=True)
    
    # 4.2 Search top_k relevant chunks
    D, I = index.search(query_vec, top_k)
    
    # 4.3 Get retrieved texts
    retrieved_chunks = [texts[i] for i in I[0]]
    
    # 4.4 Join context
    context = "\n---\n".join(retrieved_chunks)
    
    # 4.5 Construct prompt in English
    prompt = f"""You are an intelligent QA assistant. Please answer the user's question based on the following background knowledge:

Background documents:
{context}

User question:
{query}

Answer:"""
    
    # 4.6 Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # 4.7 Generate answer with sampling for diversity
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
    )
    
    # 4.8 Decode output
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 4.9 Extract answer only
    answer = generated_text[len(prompt):].strip()
    
    print("\n🧠 Answer:\n", answer)
    return answer


# 5. Command line interaction
if __name__ == "__main__":
    print("💬 Please enter your question (type 'exit' to quit)")
    while True:
        query = input("\nYour question: ")
        if query.strip().lower() in ["exit", "quit"]:
            break
        answer_question(query)


C:\Users\15278\AppData\Local\Temp\ipykernel_31888\715190709.py:5: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.3: Fast Qwen2 patching. Transformers: 4.53.2.
   \\   /|    NVIDIA GeForce RTX 4070 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
💬 Please enter your question (type 'exit' to quit)



Your question:  As a Java Developer, how can I switch my career to Machine Learning?



🧠 Answer:
 Switching from Java development to a career in machine learning (ML) is a great choice, as your programming skills will be valuable in this field. Here’s a step-by-step guide to help you make the transition: 1. Understand the Basics of Machine LearningConcepts: Familiarize yourself with key concepts such as supervised vs. unsupervised learning, regression, classification, clustering, and neural networks.Mathematics: Brush up on essential mathematics, particularly linear algebra, statistics, and calculus, which are foundational for understanding ML algorithms.2. Learn PythonWhile Java is a powerful language, Python is the most widely used language in ML due to its simplicity and the availability of numerous libraries. Start learning Python if you haven't already.3. Get Hands-On with ML LibrariesPopular Libraries: Familiarize yourself with libraries such as:Scikit-learn: For traditional ML algorithms.TensorFlow and Keras: For deep learning.PyTorch: Another popular deep learni


Your question:  exit


In [20]:
import random
import numpy as np
import faiss
import pickle

# 假设 texts 和 embeddings 已经准备好了

# 1. 打乱索引顺序
indices = list(range(len(texts)))
random.shuffle(indices)

# 2. 按比例切分
train_ratio = 0.8
train_size = int(len(texts) * train_ratio)

train_indices = indices[:train_size]
test_indices = indices[train_size:]

# 3. 分别获取 train 和 test 的文本和向量
train_texts = [texts[i] for i in train_indices]
test_texts = [texts[i] for i in test_indices]

train_embeddings = embeddings[train_indices]
test_embeddings = embeddings[test_indices]

# 4. 保存文本
with open("train_texts.pkl", "wb") as f:
    pickle.dump(train_texts, f)
with open("test_texts.pkl", "wb") as f:
    pickle.dump(test_texts, f)

# 5. 建立并保存对应的FAISS索引
dimension = embeddings.shape[1]

train_index = faiss.IndexFlatL2(dimension)
train_index.add(train_embeddings)
faiss.write_index(train_index, "train_index.faiss")

test_index = faiss.IndexFlatL2(dimension)
test_index.add(test_embeddings)
faiss.write_index(test_index, "test_index.faiss")

print(f"训练集大小: {len(train_texts)}，测试集大小: {len(test_texts)}")
print("✅ 划分完成并保存！")


训练集大小: 8070，测试集大小: 2018
✅ 划分完成并保存！


In [22]:
import pickle
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import faiss
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch

# 加载测试集文本
with open("test_texts.pkl", "rb") as f:
    test_texts = pickle.load(f)

# 初始化embedding模型（与你QA时用的同一个）
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 加载FAISS索引和原始文本库（供检索上下文）
index = faiss.read_index("qa_index.faiss")
with open("qa_chunks.pkl", "rb") as f:
    texts = pickle.load(f)

# 加载本地Qwen2模型（4bit）
model_name = "unsloth/qwen2-1.5b-bnb-4bit"
max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True
)
model.eval()

device = "cuda" if torch.cuda.is_available() else "cpu"


def answer_question(query, top_k=3):
    # 4.1 Encode query
    query_vec = embedder.encode([query], convert_to_numpy=True)
    
    # 4.2 Search top_k relevant chunks
    D, I = index.search(query_vec, top_k)
    
    # 4.3 Get retrieved texts
    retrieved_chunks = [texts[i] for i in I[0]]
    
    # 4.4 Join context
    context = "\n---\n".join(retrieved_chunks)
    
    # 4.5 Construct prompt in English
    prompt = f"""You are an intelligent QA assistant. Please answer the user's question based on the following background knowledge:

Background documents:
{context}

User question:
{query}

Answer:"""
    
    # 4.6 Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # 4.7 Generate answer with sampling for diversity
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
    )
    
    # 4.8 Decode output
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 4.9 Extract answer only
    answer = generated_text[len(prompt):].strip()
    return answer

# 从 Q&A文本块中提取问题和标准答案（支持多行答案）
def extract_qa(text):
    q_match = re.search(r"Q:\s*(.*)", text)
    a_match = re.search(r"A:\s*(.*)", text, re.DOTALL)
    question = q_match.group(1).strip() if q_match else ""
    answer = a_match.group(1).strip() if a_match else ""
    return question, answer

def evaluate_accuracy(test_texts, threshold=0.7, max_samples=200):
    total = min(len(test_texts), max_samples)
    correct = 0

    for i, text in enumerate(test_texts[:total]):
        question, true_answer = extract_qa(text)
        if not question or not true_answer:
            print(f"跳过样本 {i+1}，缺少问题或答案。")
            continue
        
        pred_answer = answer_question(question)
        if not pred_answer:
            print(f"Warning: Empty prediction for query: {question}")
            pred_answer = ""

        try:
            true_vec = embedder.encode([true_answer], convert_to_numpy=True)
            pred_vec = embedder.encode([pred_answer], convert_to_numpy=True)
            sim = cosine_similarity(true_vec, pred_vec)[0][0]
        except Exception as e:
            print(f"Error calculating similarity for sample {i+1}: {e}")
            sim = 0.0

        print(f"样本 {i+1}/{total}")
        print(f"问题: {question}")
        print(f"标准答案: {true_answer}")
        print(f"模型答案: {pred_answer}")
        print(f"相似度: {sim:.3f}")
        print("-" * 40)

        if sim >= threshold:
            correct += 1

    accuracy = correct / total if total > 0 else 0
    print(f"✅ 测试集准确率（前{total}条）: {accuracy*100:.2f}% （相似度阈值: {threshold}）")

if __name__ == "__main__":
    evaluate_accuracy(test_texts, threshold=0.7, max_samples=200)


==((====))==  Unsloth 2025.7.3: Fast Qwen2 patching. Transformers: 4.53.2.
   \\   /|    NVIDIA GeForce RTX 4070 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Input IDs of length 2734 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of length 3086 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.


样本 1/200
问题: What do software developers, engineers and/or programmers do during times of massive layoffs such as the dot-com bust or the 2008 recession?
标准答案: The good ones survived. I am one of them and lived through both periods. The Dot Com bust we saw coming. The ridiculousness of that time in hiring was palpable. People salivating over resumes that could spell HTML and people getting hired if they had a hint of an ability to use a text editor and make a web page appear in a browser. Everyone with a semi-functioning brain was buying “Learn HTML in 24 Hours” and having recruiters fall over themselves to offer amazing salaries for no demonstration of ability other than claim buzzwords on a resume. Those of us who had been writing code for years, befContinue ReadingThe good ones survived. I am one of them and lived through both periods. The Dot Com bust we saw coming. The ridiculousness of that time in hiring was palpable. People salivating over resumes that could spell HTML and peop

Unsloth: Input IDs of length 2375 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.


样本 32/200
问题: What are some good whiteboard-based interview questions for screening engineering talent at a startup?
标准答案: A good interview is a conversation where both sides do their best to figure out if the opportunity is a fit. Asking questions is essential for figuring that out. Your purpose in asking questions is to get a complete picture of the company and role. The more you know, the more likely you are to make the right decision. The questions you ask will vary depending upon what role you are interviewing for and who you are talking to. A salesperson will want to know different things than an engineer. And you will vary the questions you ask to the founding CEO vs. a recruiter who just joined the company Continue ReadingA good interview is a conversation where both sides do their best to figure out if the opportunity is a fit. Asking questions is essential for figuring that out. Your purpose in asking questions is to get a complete picture of the company and role. The more yo

...

样本 174/200
问题: What tech companies hire first year computer science students as interns?
标准答案: When I was hiring interns it was during the 1980s and 1990s. Our process was not as nearly as complicated as these other guys describe. And we were a division of a big name tech company, Control Data Corporation and then Siemens. We chose interns using basically the following check list. All interns were few as possible full time employees of the company. We did not hire interns to provide them with free training and experience.Must be college junior majoring in Software Engineering or Computer Science.Attend a local college with a program with a good reputation. We were located in a MinneapolContinue ReadingWhen I was hiring interns it was during the 1980s and 1990s. Our process was not as nearly as complicated as these other guys describe. And we were a division of a big name tech company, Control Data Corporation and then Siemens. We chose interns using basically the following check list. 

Unsloth: Input IDs of length 2424 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.


样本 182/200
问题: Should I focus on my career to be an artificial intelligence engineer, machine learning engineer, or data scientist?
标准答案: The top job in all of AI is the machine learning engineer. Almost all the jobs incorrectly labeled data scientist are really jobs for machine learning engineers. Let’s peruse a few real-world open roles. MongoDB, MySQL, AWS… not any data scientist I’ve ever met. This job is for a machine learning engineer.Another machine learning engineering role.Another mislabeled role. This is again for a machine learning engineer.Trust me on this one. The top jobs for a long time will be the machine learning engineer and data engineer.Continue ReadingThe top job in all of AI is the machine learning engineer. Almost all the jobs incorrectly labeled data scientist are really jobs for machine learning engineers. Let’s peruse a few real-world open roles. MongoDB, MySQL, AWS… not any data scientist I’ve ever met. This job is for a machine learning engineer.Another mach

Unsloth: Input IDs of length 2109 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.


样本 183/200
问题: When starting a new software engineering job, what are some good tips to set your career growth in the right direction with your manager and team?
标准答案: Starting a new software engineering job is an exciting opportunity, and setting a solid foundation for your career growth is essential. Here are some tips to help you navigate this process effectively with your manager and team: 1. Establish Clear GoalsDiscuss Career Aspirations: Have an open conversation with your manager about your career goals and interests. This can help align your work with your aspirations.Set Short- and Long-Term Goals: Define specific, measurable goals for your role and discuss them with your manager. This can include technical skills, project milestones, or leadership Continue ReadingStarting a new software engineering job is an exciting opportunity, and setting a solid foundation for your career growth is essential. Here are some tips to help you navigate this process effectively with your mana